In [5]:
import pandas as pd
import os
import re
root_path  = '.././mi_swin/subdataset'
subjects_info = pd.read_csv(os.path.join(root_path, 'participants.tsv'), sep='\t')

In [9]:
print(subjects_info)

   Participant_ID  Gender  Age  Duration ParalysisSide Handedness IsFirstTime  \
0          sub-01    male   45         1         right      right         yes   
1          sub-02    male   60         2          left      right         yes   
2          sub-03    male   60         2          left      right          no   
3          sub-04    male   56        14         right      right         yes   
4          sub-05  female   44         4          left      right         yes   
5          sub-06    male   66         6          left       left          no   
6          sub-07    male   62         5         right      right          no   
7          sub-08    male   64         5          left      right         yes   
8          sub-09    male   57         3         right      right         yes   
9          sub-10    male   55         2          left      right          no   
10         sub-11    male   31         7          left      right         yes   
11         sub-12    male   

In [7]:
subjects_info.loc[subjects_info['Participant_ID'] == 'sub-17', 'ParalysisSide'].values[0]

'left'

In [14]:
subjects_info.loc[16, 'ParalysisSide']

'left'

In [1]:
from strokes import StrokePatientsMIDataset
from torcheeg.transforms import ToTensor, Select, Compose, BandSignal
from downsample import SetSamplingRate
from baseline import BaselineCorrection
dataset = StrokePatientsMIDataset(root_path='./subdataset',
                                  io_path='.torcheeg/datasets_1736138124386_Ryk8X',
                                chunk_size=500,  # 1 second
                                overlap = 0,
                                offline_transform=Compose(
                                        [BaselineCorrection(),SetSamplingRate(500,484),
                                        BandSignal(sampling_rate=484,band_dict={'frequency_range':[8,40]})]),
                                online_transform=Compose(
                                        [ToTensor()]),
                                label_transform=Select('label'),
                                num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,500)
print(dataset[0][1], dataset[300][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-01-08 16:35:28] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1736138124386_Ryk8X.
[2025-01-08 16:35:28] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 2/2 [00:00<00:00, 168.56it/s]

[RECORD ./subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 0it [00:00, ?it/s]
[RECORD ./subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 1it [00:00,  4.32it/s]
[RECORD ./subdataset/sourcedata/sub-45/sub-45_task-motor-imagery_eeg.mat]: 2it [00:00,  5.70it/s]
[RECORD ./subdataset/sourcedata/s

torch.Size([1, 30, 484])
(0, 1) (1, 0)
320


In [3]:
print(dataset[0][0][:,17,:])  # label (int)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class EEGGenerator(nn.Module):
    def __init__(self, latent_dim, out_shape):
        super(EEGGenerator, self).__init__()
        self.latent_dim = latent_dim
        self.out_shape = out_shape
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, int(torch.prod(torch.tensor(out_shape)))),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.fc(z)
        return out.view(out.size(0), *self.out_shape)

        
class EEGDiscriminator(nn.Module):
    def __init__(self, in_shape):
        super(EEGDiscriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(in_shape))), 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_flat = x.view(x.size(0), -1)
        return self.fc(x_flat)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 超参数
latent_dim = 100
out_shape = (4, 9, 9)  # EEG 数据形状
img_size = 9
patch_size = 3
in_chans = 4
num_classes = 2

# 模型实例化
generator = EEGGenerator(latent_dim, out_shape).to(device)
discriminator = EEGDiscriminator(out_shape).to(device)

# 优化器
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 损失函数
adversarial_loss = nn.BCELoss()


In [10]:
def train_gan(dataloader, num_epochs=100):
    for epoch in range(num_epochs):
        for real_data, _ in dataloader:
            real_data = real_data.to(device)
            batch_size = real_data.size(0)

            # 真实和伪造标签
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # 生成伪造数据
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_data = generator(z)

            # 训练判别器
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_data), real_labels)
            fake_loss = adversarial_loss(discriminator(fake_data.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()

            # 训练生成器
            optimizer_G.zero_grad()
            g_loss = adversarial_loss(discriminator(fake_data), real_labels)
            g_loss.backward()
            optimizer_G.step()

        print(f"Epoch {epoch}: D Loss={d_loss.item()}, G Loss={g_loss.item()}")


In [7]:

from torch.utils.data import DataLoader, TensorDataset

# 模拟数据加载
def load_data(batch_size=32):
    num_samples = 1000
    num_channels = 4
    height, width = 9, 9
    num_classes = 2

    # EEG 数据和标签
    X = torch.randn(num_samples, num_channels, height, width)
    y = torch.randint(0, num_classes, (num_samples,))

    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader




In [11]:
dataloader = load_data(batch_size=32)
train_gan(dataloader, num_epochs=100)

Epoch 0: D Loss=1.082573413848877, G Loss=0.6837856769561768
Epoch 1: D Loss=0.876079797744751, G Loss=0.7358911633491516
Epoch 2: D Loss=0.9179106950759888, G Loss=0.69416344165802
Epoch 3: D Loss=0.8572245836257935, G Loss=0.8611480593681335
Epoch 4: D Loss=0.7106470465660095, G Loss=0.9894261360168457
Epoch 5: D Loss=0.8765276670455933, G Loss=1.1571409702301025
Epoch 6: D Loss=0.4693183898925781, G Loss=1.2732031345367432
Epoch 7: D Loss=0.43944960832595825, G Loss=1.3631253242492676
Epoch 8: D Loss=0.32611215114593506, G Loss=1.5502851009368896
Epoch 9: D Loss=0.2853534519672394, G Loss=1.9488415718078613
Epoch 10: D Loss=0.31935247778892517, G Loss=2.005789279937744
Epoch 11: D Loss=0.2779654562473297, G Loss=2.2439119815826416
Epoch 12: D Loss=0.20172058045864105, G Loss=2.6783809661865234
Epoch 13: D Loss=0.16999024152755737, G Loss=2.6512556076049805
Epoch 14: D Loss=0.09932852536439896, G Loss=3.2326254844665527
Epoch 15: D Loss=0.19447708129882812, G Loss=3.582468271255493
E

In [3]:
from eca import ECA
from model import SwinTransformer
import torch
import torch.nn as nn

class ECASwinTransformerModel(nn.Module):
    def __init__(self, num_channels):
        super(ECASwinTransformerModel, self).__init__()
        self.eca = ECA(input_size=484,num_attention_heads=1,hidden_size=484,hidden_dropout_prob=0.5)
        self.swin_transformer = SwinTransformer(
                                                in_chans=30,
                                                num_classes=2,
                                                embed_dim=192,
                                                depths=(2, 2, 18, 2),
                                                num_heads=(6, 12, 24, 48)
                                                )

    def forward(self, x):
        x = x.view(x.size(0),30,484)
        x = self.eca(x)
        x = x.view(x.size(0),30,22,22)
        x = self.swin_transformer(x)
        return x

In [4]:
import sys
import os

# 将上上级目录添加到 sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))


In [8]:
from codes.DualClassifierTrainer import DLClassifierTrainer

In [6]:
from codes.DualGanTrainer import DFTrainer

In [9]:
from torcheeg.model_selection import KFoldPerSubject
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from model import SwinTransformer
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubject(
    n_splits=5,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    model = ECASwinTransformerModel(num_channels=30)

    training_loader = DataLoader(training_dataset,
                             batch_size=32,
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=32,
                             shuffle=False)

    trainer = DLClassifierTrainer(
        model=model,
        num_classes=2,
        lr=0.001,
        weight_decay=0.01,
        devices=1,
        accelerator="gpu",
        metrics=["accuracy"]
    )

    # Training and validation
    trainer.fit(training_loader, test_loader, max_epochs=10)

    # Testing
    trainer.test(test_loader)
    
    
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-01-06 12:46:57] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-01-06 12:46:57] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1736138817091_FCMRo for the next run, if you want to use the same setting for the experiment.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /root/autodl-tmp/.autodl/kinlaw/mi_swin/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/optim/adam.py:81: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)

  | Name          | Type                    | Params
----------------------------------------------------------
0 | model1    

Sanity Checking: 0it [00:00, ?it/s]{'clip_id': '2_2', 'label': 0, 'trial_id': 2, 'baseline_id': '2_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gender': 'male', 'Age': 64, 'Duration': 30, 'ParalysisSide_y': 'right', 'Handedness': 'right', 'IsFirstTime': 'yes', 'StrokeLocation': 'Left pons', 'NIHSS': 6, 'MBI': 84, 'mRS': 4}
{'clip_id': '3_1', 'label': 1, 'trial_id': 3, 'baseline_id': '3_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gender': 'male', 'Age': 64, 'Duration': 30, 'ParalysisSide_y': 'right', 'Handedness': 'right', 'IsFirstTime': 'yes', 'StrokeLocation': 'Left pons', 'NIHSS': 6, 'MBI': 84, 'mRS': 4}
{'clip_id': '3_4', 'label': 1, 'trial_id': 3, 'baseline_id': '3_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gender': 'male', 'Age': 64, 'Duration': 30, 'ParalysisSide_y': 'right', 'Handedness': 'right'

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]

[2025-01-06 12:47:03] INFO (torcheeg/MainThread) 
[Val] val_loss: 0.856 val_accuracy: 0.438 



...

Epoch 0: 100%|██████████| 5/5 [00:00<00:00,  8.35it/s, loss=4.72, v_num=6, train_loss=4.110, train_accuracy=0.625]

[2025-01-06 12:48:25] INFO (torcheeg/MainThread) 
[Train] train_loss: 4.724 train_accuracy: 0.578 



Epoch 1:   0%|          | 0/5 [00:00<?, ?it/s, loss=4.72, v_num=6, train_loss=4.110, train_accuracy=0.625]        {'clip_id': '28_4', 'label': 0, 'trial_id': 28, 'baseline_id': '28_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gender': 'male', 'Age': 64, 'Duration': 30, 'ParalysisSide_y': 'right', 'Handedness': 'right', 'IsFirstTime': 'yes', 'StrokeLocation': 'Left pons', 'NIHSS': 6, 'MBI': 84, 'mRS': 4}
{'clip_id': '35_3', 'label': 1, 'trial_id': 35, 'baseline_id': '35_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gender': 'male', 'Age': 64, 'Duration': 30, 'ParalysisSide_y': 'right', 'Handedness': 'right', 'IsFirstTime': 'yes', 'StrokeLocation': 'Left pons', 'NIHSS': 6, 'MBI': 84, 'mRS': 4}
{'clip_id': '35_4', 'label': 1, 'trial_id': 35, 'baseline_id': '35_0', 'subject_id': 45, 'ParalysisSide_x': 'right', '_record_id': '_record_0', 'Participant_ID': 'sub-45', 'Gende

[2025-01-06 12:48:26] INFO (torcheeg/MainThread) 
[Val] val_loss: 1.327 val_accuracy: 0.656 



Epoch 6: 100%|██████████| 5/5 [00:11<00:00,  2.40s/it, loss=1.71, v_num=3, train_loss=0.994, train_accuracy=0.562]


In [ ]:
trainer = DLClassifierTrainer(
    model=model,
    num_classes=3,
    lr=0.001,
    weight_decay=0.01,
    devices=1,
    accelerator="gpu",
    metrics=["accuracy", "f1score"]
)

# Training and validation
trainer.fit(train_loader, val_loader, max_epochs=50)

# Testing
trainer.test(test_loader)

In [19]:
import torch
import torch.nn as nn

class EEGGenerator(nn.Module):
    def __init__(self, noise_dim, channels, time):
        super(EEGGenerator, self).__init__()
        self.channels = channels
        self.time = time
        self.output_dim = channels * time
        
        self.net = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, self.output_dim),
            nn.Tanh()  # 保证输出范围为 [-1, 1]
        )
    
    def forward(self, z):
        # 展平后生成
        flat_output = self.net(z)
        # 恢复到 (batch_size, channels, time)
        return flat_output.view(-1, self.channels, self.time)

# 示例用法
noise_dim = 100
channels = 32
time = 4
generator = EEGGenerator(noise_dim, channels, time)

z = torch.randn((16, noise_dim))  # 批量大小为 16 的随机噪声
fake_eeg = generator(z)           # 生成 EEG 数据，形状为 (16, 32, 4)
print(fake_eeg.shape)             # 输出: torch.Size([16, 32, 4])


torch.Size([16, 32, 4])


In [7]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim, input_channels, input_length):
        super().__init__()
        self.latent_to_features = nn.Linear(latent_dim, input_channels * input_length)
        self.reshape = lambda x: x.view(-1, input_channels, input_length)

    def forward(self, x):
        x = self.latent_to_features(x)
        return self.reshape(x)

latent_dim = 20
input_channels = 30
input_length = 484

generator1 = Generator(latent_dim, input_channels, input_length)
generator2 = Generator(latent_dim, input_channels, input_length)

class Discriminator(nn.Module):
    def __init__(self, input_channels, input_length):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_channels * input_length, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # 输出一个标量
        )

    def forward(self, x):
        return self.net(x)

discriminator1 = Discriminator(input_channels, input_length)
discriminator2 = Discriminator(input_channels, input_length)


In [11]:
trainer = DFTrainer(
    generator1=generator1,
    generator2=generator2,
    discriminator1=discriminator1,
    discriminator2=discriminator2,
    latent_channels=20,
    generator_lr=1e-4,
    discriminator_lr=1e-4,
    consistent_loss_lr=20,
    log_path="./logs",
    show_imgs=False,
    devices=1,  # 设置为你的设备数
    accelerator="gpu" if torch.cuda.is_available() else "cpu"
)


In [14]:
from torcheeg.model_selection import KFoldPerSubject
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from model import SwinTransformer
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubject(
    n_splits=5,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
   
    training_loader = DataLoader(training_dataset,
                             batch_size=32,
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=32,
                             shuffle=False)

    trainer = DFTrainer(
        generator1=generator1,
        generator2=generator2,
        discriminator1=discriminator1,
        discriminator2=discriminator2,
        latent_channels=20,
        generator_lr=1e-4,
        discriminator_lr=1e-4,
        consistent_loss_lr=20,
        log_path="./logs",
        show_imgs=False,
        devices=1,  # 设置为你的设备数
        accelerator="gpu" if torch.cuda.is_available() else "cpu"
    )
    trainer.fit(
            training_loader,
            test_loader,
            max_epochs=10
        )
    
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-01-06 13:07:54] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-01-06 13:07:54] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1736140074996_G5AN5 for the next run, if you want to use the same setting for the experiment.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /root/autodl-tmp/.autodl/kinlaw/mi_swin/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type              | Params
----------------------------------------------------------
0  | generator1         | Generator         | 304 K 
1  | discriminator1     | Discriminator     | 1.9 M 
2  | generator2         | Generator         | 304 K 
3  | discriminator2     | Discriminator     | 1.9 M 
4  | bce_fn             | BCEWithLogitsLoss | 0     
5  | consistent_loss_fn | L1Loss            | 0     
6  | t

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: mat1 and mat2 shapes cannot be multiplied (960x484 and 20x14520)